In [1]:
import os.path
from multiprocessing import Pool
import sys
import SimpleITK as sitk
import torch
from batchgenerators.utilities.file_and_folder_operations import nifti_files, join, maybe_mkdir_p, isdir
sys.stdout = open(os.devnull, 'w')
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
sys.stdout = sys.__stdout__
from HD_BET.paths import folder_with_parameter_files


In [39]:

# def dilate_mask(mask, iterations=1):

def apply_bet(img, bet, out_fname, dilation_radius):
    """
    Apply BET mask to image and save the result.
    """
    img_itk = sitk.ReadImage(img)
    img_npy = sitk.GetArrayFromImage(img_itk)
    img_bet = sitk.ReadImage(bet)
    dilated_bet = sitk.BinaryDilate(img_bet, [int(dilation_radius)]*3)
    img_bet_npy = sitk.GetArrayFromImage(dilated_bet)
    img_npy[img_bet_npy == 0] = 0
    out = sitk.GetImageFromArray(img_npy)
    out.CopyInformation(img_itk)
    sitk.WriteImage(out, out_fname)


In [3]:
def get_hdbet_predictor(
        use_tta: bool = False,
        device: torch.device = torch.device('cuda'),
        verbose: bool = False
):
    os.environ['nnUNet_compile'] = 'F'
    predictor = nnUNetPredictor(
        tile_step_size=0.5,
        use_gaussian=True,
        use_mirroring=use_tta,
        perform_everything_on_device=True,
        device=device,
        verbose=verbose,
        verbose_preprocessing=verbose
    )
    predictor.initialize_from_trained_model_folder(
        folder_with_parameter_files,
        'all'
    )
    if device == torch.device('cpu'):
        torch.set_num_threads(os.cpu_count())
    return predictor


In [ ]:
def hdbet_predict(
        input_file_or_folder: str,
        output_file_or_folder: str,
        predictor: nnUNetPredictor,
        keep_brain_mask: bool = False,
        compute_brain_extracted_image: bool = True,
        dilation_radius: int = 5,
):
    # find input file or files
    if os.path.isdir(input_file_or_folder):
        input_files = nifti_files(input_file_or_folder)
        # output_file_or_folder must be folder in this case
        maybe_mkdir_p(output_file_or_folder)
        output_files = [join(output_file_or_folder, os.path.basename(i)) for i in input_files]
        brain_mask_files = [i[:-7] + '_bet.nii.gz' for i in output_files]
    else:
        assert not isdir(output_file_or_folder), 'If input is a single file then output must be a filename, not a directory'
        assert output_file_or_folder.endswith('.nii.gz'), 'Output file must end with .nii.gz'
        input_files = [input_file_or_folder]
        output_files = [join(os.path.curdir, output_file_or_folder)]
        brain_mask_files = [join(os.path.curdir, output_file_or_folder[:-7] + '_bet.nii.gz')]

    # we first just predict the brain masks using the standard nnU-Net inference
    predictor.predict_from_files(
        [[i] for i in input_files],
        brain_mask_files,
        save_probabilities=False,
        overwrite=True,
        num_processes_preprocessing=4,
        num_processes_segmentation_export=8,
        folder_with_segs_from_prev_stage=None,
        num_parts=1,
        part_id=0
    )
    # remove unnecessary json files
    os.remove(join(os.path.dirname(brain_mask_files[0]), 'dataset.json'))
    os.remove(join(os.path.dirname(brain_mask_files[0]), 'plans.json'))
    os.remove(join(os.path.dirname(brain_mask_files[0]), 'predict_from_raw_data_args.json'))

    if compute_brain_extracted_image:
        # now brain extract the images
        res = []
        with Pool(4) as p:
            for im, bet, out in zip(input_files, brain_mask_files, output_files):
                res.append(
                    p.starmap_async(
                    apply_bet,
                    ((im, bet, out, dilation_radius),)
                    )
                )
            [i.get() for i in res]

    if not keep_brain_mask:
        [os.remove(i) for i in brain_mask_files]
    # return output_files, brain_mask_files

In [5]:
import torch

from HD_BET.checkpoint_download import maybe_download_parameters
from HD_BET.hd_bet_prediction import get_hdbet_predictor, hdbet_predict

In [ ]:
# def main():
#     print("\n########################")
#     print("If you are using hd-bet, please cite the following papers:\n")
#     print("Isensee F, Schell M, Tursunova I, Brugnara G, Bonekamp D, Neuberger U, Wick A, Schlemmer HP, Heiland S, Wick W, "
#            "Bendszus M, Maier-Hein KH, Kickingereder P. Automated brain extraction of multi-sequence MRI using artificial "
#            "neural networks. arXiv preprint arXiv:1901.11341, 2019.\n")
#     print(
#         "Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). "
#         "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. "
#         "Nature methods, 18(2), 203-211.")

#     print("########################\n")

#     import argparse
#     parser = argparse.ArgumentParser()
#     parser.add_argument('-i', '--input', help='input. Can be either a single file name or an input folder. If file: must be '
#                                        'nifti (.nii.gz) and can only be 3D. No support for 4d images, use fslsplit to '
#                                        'split 4d sequences into 3d images. If folder: all files ending with .nii.gz '
#                                        'within that folder will be brain extracted.', required=True, type=str)
#     parser.add_argument('-o', '--output', help='output. Can be either a filename or a folder. If it does not exist, the folder'
#                                      ' will be created', required=False, type=str)
#     parser.add_argument('-device', default='cuda', type=str, required=False,
#                         help='used to set on which device the prediction will run. Can be \'cuda\' (=GPU), \'cpu\' or '
#                              '\'mps\'. Default: cuda')
#     parser.add_argument('--disable_tta', required=False, action='store_true',
#                         help='Set this flag to disable test time augmentation. This will make prediction faster at a '
#                              'slight decrease in prediction quality. Recommended for device cpu')

#     parser.add_argument('--save_bet_mask', action='store_true', required=False,
#                         help='Set this flag to keep the bet masks. Otherwise they will be removed once HD_BET is '
#                              'done')
#     parser.add_argument('--no_bet_image', action='store_true', required=False,
#                         help="Set this flag to disable generating the skull stripped/brain extracted image. Only "
#                              "makes sense if you also set --save_bet_mask")
#     parser.add_argument('--verbose', action='store_true', required=False,
#                         help="Talk to me.")

#     args = parser.parse_args()

#     maybe_download_parameters()

#     predictor = get_hdbet_predictor(
#         use_tta=not args.disable_tta,
#         device=torch.device(args.device),
#         verbose=args.verbose
#     )

#     hdbet_predict(args.input,args.output, predictor, keep_brain_mask=args.save_bet_mask,
#                   compute_brain_extracted_image=not args.no_bet_image)


# if __name__ == '__main__':
#     main()

In [6]:
maybe_download_parameters()

predictor = get_hdbet_predictor(
    use_tta=not False,
    device=torch.device('cuda'),
    verbose=True
)

In [64]:
dilatesize = 10 ## mm
input_file = "/data_dzne_archiv2/Studien/Deep_Learning_Visualization/temporary_stuff/Jaya_Chandra_Terli/Brain_extraction/hd_bet_output/20160331092323_Rostock_DZNE_BRAIN_F_s216320.nii.gz"
output_file = f"/data_dzne_archiv2/Studien/Deep_Learning_Visualization/temporary_stuff/Jaya_Chandra_Terli/Brain_extraction/hd_bet_output/dilate_{dilatesize}mm_20160331092323_Rostock_DZNE_BRAIN_F_s216320.nii.gz"

In [54]:
#for 1cm=10mm of bet-mask boundary expansion, we are
dilation_radius = 10 / 2 #dilating 5 voxels in each direction

In [65]:
hdbet_predict(
    input_file,
    output_file, 
    predictor, 
    keep_brain_mask=True,
    compute_brain_extracted_image=not False,
    dilation_radius=dilation_radius
    )


100%|██████████| 12/12 [00:06<00:00,  1.93it/s]


(['/data_dzne_archiv2/Studien/Deep_Learning_Visualization/temporary_stuff/Jaya_Chandra_Terli/Brain_extraction/hd_bet_output/dilate_10mm_20160331092323_Rostock_DZNE_BRAIN_F_s216320.nii.gz'],
 ['/data_dzne_archiv2/Studien/Deep_Learning_Visualization/temporary_stuff/Jaya_Chandra_Terli/Brain_extraction/hd_bet_output/dilate_10mm_20160331092323_Rostock_DZNE_BRAIN_F_s216320_bet.nii.gz'])

In [ ]:
# img_itk = sitk.ReadImage(input_file)
# img_npy = sitk.GetArrayFromImage(img_itk)
# img_bet = sitk.ReadImage(masks[0])
# img_bet.GetSpacing() # 1 voxel size: (x-mm, y-mm, z-mm)

In [66]:
pet_input = "/mnt/mrt/Describe/_downloaded/Muttray_FDG-PET/0050311816 MUTTRAY GTZ/2930177 PET FDG_Hirn/PT BR_CTAC Dynamic Brain/"

In [69]:
dcm_0 = pet_input + "PT000000.dcm"

In [71]:
img_dcm = sitk.ReadImage(dcm_0)

In [75]:
img_dcm.GetSpacing()

(2.0, 2.0, 1.0)